# Notebook: Fine-tune BERT on Your Labeled CSV Data

### Objective:

1. Walk through the entire fine-tuning pipeline step by step
2. Explain what's going on in accessible language
3. Prints out shapes, label distrubtions, and smaple rows at each stage so you can "see" the data evolving.
4. Uses the `local_llm`'s core functions, so the notebook stays relatively clean and focused on _workflow_, not implementation details.

### Step 1 - Setup and Imports 

In this fist step, we import all the tools we need:
- `pandas` to load and inspect the CSV.
- The fine-tuning helpers form `local_llm.training.text_finetune`.

We'll also define the paths to your data and BERT assets later. 

In [1]:
from pathlib import Path
import pandas as pd

from local_llm.training.text_finetune import (
    FineTuneConfig,
    set_seed,
    prepare_label_mapping,
    stratified_split_indices,
    encode_splits,
    build_dataloaders,
    build_bert_text_classifier_from_assets,
    train_text_classifier,
    evaluate_on_split,
    save_finetuned_classifier,
    export_predictions_csv,
)

print("Imports complete.")


Imports complete.


### Step 2 - Load Your Labeled Data

Now we: 
1. Load the labeled CSV file into a pandas DataFrame.
2. Print:
    - The first few rows
    - Shape of the data (rows, columns)
    - The Column Names
 
This helps confirm that:
- The file path is correct.
- The column names match what we expect later.

In [2]:
data_path = Path("C:/Users/Cameron.Webster/Python/local-llm/data/wbs_training_data.csv")  # adjust if your file lives elsewhere

print(f"Loading data from: {data_path.resolve()}")
df = pd.read_csv(data_path)

print("\n✅ Data loaded.")
print(f"Data shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
print("Columns:")
print(df.columns.tolist())

print("\nFirst 5 rows:")
display(df.head())

Loading data from: C:\Users\Cameron.Webster\Python\local-llm\data\wbs_training_data.csv

✅ Data loaded.
Data shape: 19857 rows × 5 columns

Columns:
['parsid', 'wbs', 'wbs_name', 'keyword', 'level_1']

First 5 rows:


,parsid,wbs,wbs_name,keyword,level_1
0,389,3.1,M&O Support,m o support,OPC Testing and Startup
1,389,3.2,DOE Support,doe support,OPC Testing and Startup
2,389,2.1.0,Enhanced Conceptual Design (ECD),conceptual design,Design and NRE
3,389,2.1.1,Preliminary Design (PD),preliminary design,Design and NRE
4,389,2.1.2,Final Design (FD),final design,Design and NRE


### Step 3 - Define Fune-tuning Configuration

Here we deine a `FineTuneConfig`, which is a single object that:
- Tells the library which text columns touse and which columns is the label. 
- Controls how we split into train/validation/test sets.
- Point to your BERT assets directory. 
- Sets training hyperparameters like learning rate and number of epochs.
- Defines how much BERT we fine-tune(`finetune_policy` and `finetune_last_n`).

We also call `set_seed` to make the run reproducible (same random splits each time).

In [3]:
cfg = FineTuneConfig(
    # Which text columns to concatenate into a single input string
    text_cols=("wbs_name", "keyword"),

    # Which column contains the labels
    label_col="level_1",

    # Where the converted BERT assets live
    assets_dir=Path("C:/Users/Cameron.Webster/Python/local-llm/assets/bert-base-local"),

    # Where to save fine-tuning outputs (models, predictions, etc.)
    output_dir=Path("C:/Users/Cameron.Webster/Python/local-llm/artifacts/finetune_bert"),

    # Train/val/test fractions (they should sum to <= 1, remainder is test)
    train_frac=0.8,
    val_frac=0.10,

    # Training loop hyperparameters
    epochs=5,
    base_lr=2e-5,
    batch_size=32,

    # How much of BERT to fine-tune
    finetune_policy="last_n",  # options: "none", "last_n", "full"
    finetune_last_n=2,         # last 2 transformer layers trainable
    train_embeddings=False,    # embeddings stay frozen
)

print("FineTuneConfig created:")
print(cfg)

# Set random seed for reproducibility
set_seed(cfg.seed)
print(f"\nRandom seed set to: {cfg.seed}")


FineTuneConfig created:
FineTuneConfig(text_cols=('wbs_name', 'keyword'), label_col='level_1', train_frac=0.8, val_frac=0.1, seed=42, assets_dir=WindowsPath('C:/Users/Cameron.Webster/Python/local-llm/assets/bert-base-local'), max_len=256, lowercase=True, batch_size=32, epochs=5, base_lr=2e-05, weight_decay=0.01, grad_clip_norm=1.0, finetune_policy='last_n', finetune_last_n=2, train_embeddings=False, pooling='cls', head_config=ClassifierHeadConfig(hidden_sizes=(768,), dropouts=(0.15, 0.2), use_layer_norm=True, activation='gelu'), device='cuda', output_dir=WindowsPath('C:/Users/Cameron.Webster/Python/local-llm/artifacts/finetune_bert'), run_name='finetune_run')

Random seed set to: 42


### Step 4 - Map String Labels to Integer IDs

Neural networks operate on numbers, not strings. 

Here we:
- Convert the label columns (e.g., `"Construction"`, `"Design"`) into integers IDs.
- Build two dictionaries:
    - `label_to_id`: string label --> integer
    - `id_to_label`: integer --> string label
- Add a new column `label_id` to the DataFrame.

We also print:
- The mappings
- How many examples there are per class (class balanace)

In [4]:
label_col = cfg.label_col
print(f"Preparing label mapping using label column: '{label_col}'")

df_mapped, label_to_id, id_to_label = prepare_label_mapping(df, label_col)

print("\n✅ Label mapping created.")
print("label_to_id:")
for lab, idx in label_to_id.items():
    print(f"  {lab!r} -> {idx}")

print("\nid_to_label:")
for idx, lab in id_to_label.items():
    print(f"  {idx} -> {lab!r}")

print("\nValue counts for label_id (class distribution):")
display(df_mapped["label_id"].value_counts().sort_index())

print("\nPreview with label_id:")
display(df_mapped[[label_col, "label_id"]].head())


Preparing label mapping using label column: 'level_1'

✅ Label mapping created.
label_to_id:
  'Construction' -> 0
  'D&D' -> 1
  'Design and NRE' -> 2
  'OPC Testing and Startup' -> 3
  'Process Equipment' -> 4
  'SEPM' -> 5
  'Site Preparation' -> 6
  'Standard Equipment' -> 7

id_to_label:
  0 -> 'Construction'
  1 -> 'D&D'
  2 -> 'Design and NRE'
  3 -> 'OPC Testing and Startup'
  4 -> 'Process Equipment'
  5 -> 'SEPM'
  6 -> 'Site Preparation'
  7 -> 'Standard Equipment'

Value counts for label_id (class distribution):


label_id
0    2676
1     509
2    4112
3    1551
4    5198
5    2880
6     659
7    2272
Name: count, dtype: int64


Preview with label_id:


,level_1,label_id
0,OPC Testing and Startup,3
1,OPC Testing and Startup,3
2,Design and NRE,2
3,Design and NRE,2
4,Design and NRE,2


### Step 5 - Stratified Train / Validation / Test Split

We now split the data into three sets:
- **Training**: used to fit the model. 
- **Validation**: used to tune hyperparameters and monitor overfitting.
- **Test**: held out until the very end for final evaluation. 

The split is stratified, meaning:
- Each set keeps apprximately the same label distribution as the full dataset. 

We print:
- The sizes of each split.
- Label distributions in each split.

In [5]:
labels = df_mapped["label_id"].values

train_idx, val_idx, test_idx = stratified_split_indices(
    labels,
    train_frac=cfg.train_frac,
    val_frac=cfg.val_frac,
    seed=cfg.seed,
)

print("✅ Stratified indices created.")
print(f"Train size: {len(train_idx)}")
print(f"Val size:   {len(val_idx)}")
print(f"Test size:  {len(test_idx)}")

# Inspect label distribution across splits
train_labels = df_mapped.iloc[train_idx]["label_id"]
val_labels   = df_mapped.iloc[val_idx]["label_id"]
test_labels  = df_mapped.iloc[test_idx]["label_id"]

print("\nLabel distribution in TRAIN:")
display(train_labels.value_counts().sort_index())

print("\nLabel distribution in VAL:")
display(val_labels.value_counts().sort_index())

print("\nLabel distribution in TEST:")
display(test_labels.value_counts().sort_index())


✅ Stratified indices created.
Train size: 15886
Val size:   1986
Test size:  1985

Label distribution in TRAIN:


label_id
0    2141
1     407
2    3290
3    1241
4    4158
5    2304
6     527
7    1818
Name: count, dtype: int64


Label distribution in VAL:


label_id
0    268
1     51
2    411
3    155
4    520
5    288
6     66
7    227
Name: count, dtype: int64


Label distribution in TEST:


label_id
0    267
1     51
2    411
3    155
4    520
5    288
6     66
7    227
Name: count, dtype: int64

### Step 6 - Encode Text with BERT Input Encoder

BERT expects tokeized inputs:
- `input_ids`: integers representing word pieces.
- `attention_mask`: 1 for real tokens, 0 for padding.
- `token_type_ids`: segment IDs (here mostly 0s since we use single sentences).

We use:
- `encode_splits` to:
    - Build a tokenier based on your local BERT vocab.
    - Turn text into tensors for each split (train/validation/test).

We print:
- Tensor shapes for each split.
- Sequence length (should be equal `cfg.max_len`).

In [6]:
print("Encoding text splits using BERT input encoder...")
splits = encode_splits(df_mapped, train_idx, val_idx, test_idx, cfg)

for split_name, tensors in splits.items():
    print(f"\nSplit: {split_name}")
    for key, tensor in tensors.items():
        print(f"  {key}: shape={tuple(tensor.shape)}, dtype={tensor.dtype}")


Encoding text splits using BERT input encoder...

Split: train
  input_ids: shape=(15886, 256), dtype=torch.int64
  token_type_ids: shape=(15886, 256), dtype=torch.int64
  attention_mask: shape=(15886, 256), dtype=torch.int64
  labels: shape=(15886,), dtype=torch.int64

Split: val
  input_ids: shape=(1986, 256), dtype=torch.int64
  token_type_ids: shape=(1986, 256), dtype=torch.int64
  attention_mask: shape=(1986, 256), dtype=torch.int64
  labels: shape=(1986,), dtype=torch.int64

Split: test
  input_ids: shape=(1985, 256), dtype=torch.int64
  token_type_ids: shape=(1985, 256), dtype=torch.int64
  attention_mask: shape=(1985, 256), dtype=torch.int64
  labels: shape=(1985,), dtype=torch.int64


### Step 7 - Build PyTorch Datasets and DataLoaders

PyTorch's `DataLoader`:
- Handles batching (groups of exaqmples processed together).
- Optionally shuffles data (for training).

We:
- Wrap each split (train, validation, test) into `TensorDictDataset`.
- Build `DataLoaders`s with a batch size from `cfg.batch_size`.

We print:
- Number of batches for each split.
- The shape of one batch from the trianing loader.

In [7]:
print("Building PyTorch DataLoaders...")
loaders = build_dataloaders(splits, cfg)

for name, loader in loaders.items():
    num_batches = len(loader)
    print(f"{name.capitalize()} loader: {num_batches} batch(es) with batch_size ~ {loader.batch_size}")

# Peek at one training batch to understand the tensor shapes
train_loader = loaders["train"]
first_batch = next(iter(train_loader))

input_ids_batch, token_type_ids_batch, attention_mask_batch, labels_batch = first_batch

print("\nExample TRAIN batch shapes:")
print(f"  input_ids:      {tuple(input_ids_batch.shape)}")
print(f"  token_type_ids: {tuple(token_type_ids_batch.shape)}")
print(f"  attention_mask: {tuple(attention_mask_batch.shape)}")
print(f"  labels:         {tuple(labels_batch.shape)}")


Building PyTorch DataLoaders...
Train loader: 497 batch(es) with batch_size ~ 32
Val loader: 32 batch(es) with batch_size ~ 64
Test loader: 32 batch(es) with batch_size ~ 64

Example TRAIN batch shapes:
  input_ids:      (32, 256)
  token_type_ids: (32, 256)
  attention_mask: (32, 256)
  labels:         (32,)


### Step 8 - Load BERT and Build the Classifier

Now we:
- Load the base BERT encoder from your local assets:
    - Reads `config.json` and `pytorch_model.bin`.
- Wrap it in a `BertTextClassifier`, which:
    - Uses either `[CLS]` token or mean pooling (here: `cfg.pooling`).
    - Adds a classifier head on top to predict your label classes. 
- Apply your fine-tuning policy (e.g., train only the last N transformer layers).

We print:
- Number of labels
- model device (CPU or GPU).
- Pooling strategy.
- Fine-tuning Policy.

In [8]:
num_labels = len(label_to_id)
print(f"Building BERT text classifier for {num_labels} label(s)...")

model = build_bert_text_classifier_from_assets(cfg, num_labels=num_labels)

print("\n✅ Model built.")
print(f"Pooling strategy: {model.pooling}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Fine-tune policy: {cfg.finetune_policy}, last_n={cfg.finetune_last_n}, train_embeddings={cfg.train_embeddings}")


Building BERT text classifier for 8 label(s)...

✅ Model built.
Pooling strategy: cls
Model device: cuda:0
Fine-tune policy: last_n, last_n=2, train_embeddings=False


### Step 9 - Train the Classifier

**Time to train!**

`train_text_classifer` will:
- Build an optimizer (AdamW) using only trainable parameters.
- Loop over epochs:
    - Train on the training loader.
    - Evaluate on the validation loader
- Track:
    - Training/validation loss.
    - Training/validation accuracy.
- Keep the best model state (based on validation accuracy). 

We print:
- The per-epoch metrics (the function already does this).
- The final training history in a DataFrame.

In [9]:
print("Starting training...")
history, best_state = train_text_classifier(model, loaders, cfg)

print("\n✅ Training complete.")

# Convert history to a DataFrame for easier viewing
history_df = pd.DataFrame(history)
print("\nTraining history:")
display(history_df)


Starting training...
Epoch 01 | train_loss=0.8586 acc=0.717 | val_loss=0.3604 acc=0.886
Epoch 02 | train_loss=0.3666 acc=0.889 | val_loss=0.2311 acc=0.925
Epoch 03 | train_loss=0.2432 acc=0.926 | val_loss=0.1432 acc=0.956
Epoch 04 | train_loss=0.1779 acc=0.948 | val_loss=0.1073 acc=0.965
Epoch 05 | train_loss=0.1351 acc=0.960 | val_loss=0.1089 acc=0.970

✅ Training complete.

Training history:


,epoch,train_loss,train_acc,val_loss,val_acc
0,1,0.858552,0.716795,0.360401,0.885700
1,2,0.366604,0.888707,0.231077,0.925478
2,3,0.243237,0.926098,0.143210,0.956193
3,4,0.177934,0.947879,0.107343,0.965257
4,5,0.135092,0.959839,0.108859,0.969789


### Step 10 - Save the Fine-tuned Model and Metadata 

After training, we want to:
- Save:
    - The full classifier (encoder + head).
    - The fine-tuned encoder weights in BERT format.
    - A small JSON metadata file with label mappings and training settings. 

The make it easy to:
- Reload the model later
- Run inference in an other script or environment.

In [10]:
print("Saving fine-tuned classifier and encoder weights...")
save_finetuned_classifier(model, best_state, cfg, label_to_id, id_to_label)

print("\n✅ Fine-tuned model and metadata saved to:")
print(cfg.output_dir.resolve())


Saving fine-tuned classifier and encoder weights...

✅ Fine-tuned model and metadata saved to:
C:\Users\Cameron.Webster\Python\local-llm\artifacts\finetune_bert


### Step 11 - Evaluate on Test Set and Inspect Predictions

Now we evaluate on the test split:
- Compute:
    - Test loss
    - Test accuracy
- Collect Predictions
    - True label ids
    - predicted label ids
    - Confidence scores (max softmax probability)

Then: 
- Merge predictions back with the originial test rows.
- Save the combined table as a CSV (so you can inspect errors, etc.).

In [11]:
print("Evaluating model on TEST split...")
test_metrics, test_preds = evaluate_on_split(model, splits["test"], cfg)

print("\n✅ Test evaluation complete.")
print("Test metrics:")
for k, v in test_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\nFirst 10 prediction rows (label_id, pred_label_id, pred_confidence):")
display(test_preds.head(10))

# Align test raw rows with test_idx
test_raw_df = df_mapped.iloc[test_idx]

print("\nExporting test predictions merged with raw data to CSV...")
csv_path = export_predictions_csv(test_preds, test_raw_df, id_to_label, cfg, split_name="test")

print("\n✅ Test predictions saved to CSV:")
print(csv_path.resolve())

Evaluating model on TEST split...

✅ Test evaluation complete.
Test metrics:
  loss: 0.1018
  acc: 0.9723

First 10 prediction rows (label_id, pred_label_id, pred_confidence):


,label_id,pred_label_id,pred_confidence
0,4,4,0.996238
1,0,0,0.999057
2,1,1,0.990744
3,3,2,0.698315
4,4,4,0.968674
5,4,4,0.999564
6,4,4,0.997478
7,2,2,0.999216
8,3,3,0.996698
9,4,4,0.999688



Exporting test predictions merged with raw data to CSV...

✅ Test predictions saved to CSV:
C:\Users\Cameron.Webster\Python\local-llm\artifacts\finetune_bert\test_predictions.csv


### Step 12 - Optional Quick Sanity Check on the Output CSV

Finally, we can quickly reload the exported CSV and:
- Check a few rows
- look at the distribution of predicted labels

This helps sanity-check that:
- The file wrote correctly.
- THe predictions look reasonable.

In [12]:
print("Reloading exported predictions for sanity check...")
preds_loaded = pd.read_csv(csv_path)

print(f"\nLoaded {len(preds_loaded)} rows from predictions CSV.")
print("Columns in predictions CSV:")
print(preds_loaded.columns.tolist())

print("\nFirst 5 rows from predictions CSV:")
display(preds_loaded.head())

print("\nPredicted label counts:")
if "pred_label" in preds_loaded.columns:
    display(preds_loaded["pred_label"].value_counts())
elif "pred_label_id" in preds_loaded.columns:
    display(preds_loaded["pred_label_id"].value_counts())



Reloading exported predictions for sanity check...

Loaded 1985 rows from predictions CSV.
Columns in predictions CSV:
['parsid', 'wbs', 'wbs_name', 'keyword', 'level_1', 'label_id', 'pred_label_id', 'pred_label', 'pred_confidence']

First 5 rows from predictions CSV:


,parsid,wbs,wbs_name,keyword,level_1,label_id,pred_label_id,pred_label,pred_confidence
0,1086,1.07.02.09.45.80.20.31.25,PSF Material Handling & Processing,psf construction support facility handl materi...,Process Equipment,4,4,Process Equipment,0.996238
1,1155,11.03.04.50649,ARI-SXN Beamline Infrastructure,structure,Construction,0,0,Construction,0.999057
2,1244,3.32.08.42.02.05,OSWDF 413 Critical Decision 4 (CD - 4) ESP,disposal site prep facility construction,D&D,1,1,D&D,0.990744
3,1140,1.03.04.01,"OPR, Specification, and Design Review Reports",commissioning,OPC Testing and Startup,3,2,Design and NRE,0.698315
4,1092,A.B.06.01.02.03,Comm Support - Beamlines and Optical Systems,optical commissioning,Process Equipment,4,4,Process Equipment,0.968674



Predicted label counts:


pred_label
Process Equipment          534
Design and NRE             412
SEPM                       282
Construction               267
Standard Equipment         219
OPC Testing and Startup    150
Site Preparation            69
D&D                         52
Name: count, dtype: int64